# Optimizing the Model

### Attempts Made:
* Increasing the number of values for each bin
* Adding more neurons to a hidden layer
* Adding more hidden layers
* Adding number of epochs to the training regimen

## Preprocessing

In [18]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [21]:
# Look at APPLICATION_TYPE value counts for binning
appTypeCounts = application_df['APPLICATION_TYPE'].value_counts()
appTypeCounts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(appTypeCounts[appTypeCounts < 100].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [23]:
# Look at CLASSIFICATION value counts for binning
classCounts = application_df['CLASSIFICATION'].value_counts()
classCounts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C8210        1
C4200        1
C6100        1
C2570        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [24]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classCounts[classCounts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C3200        2
C1256        2
C1267        2
C1246        2
C1234        2
Name: CLASSIFICATION, dtype: int64

In [25]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classCounts[classCounts < 700].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [26]:
# Generate our categorical variable lists
appCategorical = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
appCategorical

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [27]:
# Create a OneHotEncoder instance
encode = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(encode.fit_transform(application_df[appCategorical]))

# Add the encoded variable names to the dataframe
encode_df.columns = encode.get_feature_names(appCategorical)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [28]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(appCategorical, 1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [29]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  90
hidden_nodes_layer2 = 45
hidden_nodes_layer3 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 90)                4140      
                                                                 
 dense_5 (Dense)             (None, 45)                4095      
                                                                 
 dense_6 (Dense)             (None, 15)                690       
                                                                 
 dense_7 (Dense)             (None, 1)                 16        
                                                                 
Total params: 8,941
Trainable params: 8,941
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5361 - accuracy: 0.7398
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5362 - accuracy: 0.7400
Epoch 3/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5360 - accuracy: 0.7400
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5359 - accuracy: 0.7406
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5356 - accuracy: 0.7401
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5363 - accuracy: 0.7406
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5355 - accuracy: 0.7404
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5354 - accuracy: 0.7401
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5352 - accuracy: 0.7407
Epoch 10/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5355 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.5337 - accuracy: 0.7412
Epoch 82/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5324 - accuracy: 0.7418
Epoch 83/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7421
Epoch 84/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5317 - accuracy: 0.7420
Epoch 85/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7420
Epoch 86/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5321 - accuracy: 0.7419
Epoch 87/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5318 - accuracy: 0.7425
Epoch 88/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7420
Epoch 89/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5314 - accuracy: 0.7419
Epoch 90/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5316 - accuracy: 

804/804 [==============================] - 2s 2ms/step - loss: 0.5302 - accuracy: 0.7425
Epoch 161/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5307 - accuracy: 0.7428
Epoch 162/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5300 - accuracy: 0.7425
Epoch 163/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5319 - accuracy: 0.7427
Epoch 164/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5302 - accuracy: 0.7427
Epoch 165/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5305 - accuracy: 0.7425
Epoch 166/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5297 - accuracy: 0.7426
Epoch 167/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5299 - accuracy: 0.7421
Epoch 168/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5298 - accuracy: 0.7426
Epoch 169/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5303 - a

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5795 - accuracy: 0.7259 - 355ms/epoch - 1ms/step
Loss: 0.5794663429260254, Accuracy: 0.7259474992752075


In [37]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')